# Tokenizers

For this Colab session, we explore the world of Tokenizers

You can run this notebook on a free CPU, or locally on your box if you prefer.


In [2]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer

In [3]:
# Log in to Hugging Face

hf_token = userdata.get('HF_TOKEN')
if hf_token and hf_token.startswith("hf_"):
  print("HF key looks good so far")
else:
  print("HF key is not set - please click the key in the left sidebar")
login(hf_token, add_to_git_credential=True)


HF key looks good so far


# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole 3.1 family of models. For whatever reason, occasionally Meta doesn't approve access. If that happens to you, please follow [this](https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing) troubleshooting.

If the next cell gives you an error, then please check:  
1. Are you logged in to HuggingFace? Try running `login()` to check your key works
2. Did you set up your API key with full read and write permissions?
3. If you visit the Llama3.1 page with the link above, does it show that you have access to the model near the top?

I've also set up this troubleshooting colab to try to diagnose any HuggingFace connectivity issues:  
https://colab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing


In [4]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B', trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [6]:
text = "HI, my name is Tanmmay"
tokens = tokenizer.encode(text)
tokens

[128000, 24860, 11, 856, 836, 374, 25566, 3906, 352]

In [7]:
character_count = len(text)
word_count = len(text.split(' '))
token_count = len(tokens)
print(f"There are {character_count} characters, {word_count} words and {token_count} tokens")

There are 22 characters, 5 words and 9 tokens


In [8]:
tokenizer.decode(tokens)


'<|begin_of_text|>HI, my name is Tanmmay'

In [9]:
tokenizer.batch_decode(tokens)

['<|begin_of_text|>', 'HI', ',', ' my', ' name', ' is', ' Tan', 'mm', 'ay']

In [10]:
# tokenizer.vocab
tokenizer.get_added_vocab()

{'<|begin_of_text|>': 128000,
 '<|end_of_text|>': 128001,
 '<|reserved_special_token_0|>': 128002,
 '<|reserved_special_token_1|>': 128003,
 '<|finetune_right_pad_id|>': 128004,
 '<|reserved_special_token_2|>': 128005,
 '<|start_header_id|>': 128006,
 '<|end_header_id|>': 128007,
 '<|eom_id|>': 128008,
 '<|eot_id|>': 128009,
 '<|python_tag|>': 128010,
 '<|reserved_special_token_3|>': 128011,
 '<|reserved_special_token_4|>': 128012,
 '<|reserved_special_token_5|>': 128013,
 '<|reserved_special_token_6|>': 128014,
 '<|reserved_special_token_7|>': 128015,
 '<|reserved_special_token_8|>': 128016,
 '<|reserved_special_token_9|>': 128017,
 '<|reserved_special_token_10|>': 128018,
 '<|reserved_special_token_11|>': 128019,
 '<|reserved_special_token_12|>': 128020,
 '<|reserved_special_token_13|>': 128021,
 '<|reserved_special_token_14|>': 128022,
 '<|reserved_special_token_15|>': 128023,
 '<|reserved_special_token_16|>': 128024,
 '<|reserved_special_token_17|>': 128025,
 '<|reserved_special_to

In [11]:
len(tokenizer.vocab)

128256

# Instruct variants of models

Many models have a variant that has been trained for use in Chats.  
These are typically labelled with the word "Instruct" at the end.  
They have been trained to expect prompts with a particular format that includes system, user and assistant prompts.  

There is a utility method `apply_chat_template` that will convert from the messages list format we are familiar with, into the right input prompt for this model.

In [12]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [15]:

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>




## Crucial "Aha" moment

For few weeks, I've given you the impression that LLMs could receive a list of python dictionaries in some way:

```python
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
```

But an LLM is just a Data Science model that takes a sequence of numbers and predicts the probability of the next number! You can't pass a bunch of Python objects into a statistical model!

### And now you have the missing piece of the puzzle..

The messages in OpenAI format get converted:

1. ...into a sequence of words with special tags to separate the System, User, Assistant prompt
2. ...then the words are broken down into fragments - "tokens"
3. ...then the tokens are replaced with Token IDs - and this is the input sequence

> The input to an LLM is a sequence of Token IDs. The output is the probability distribution of the next Token ID to follow this input.

That's it!


In [14]:

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
print(prompt)

[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 2675, 527, 264, 11190, 18328, 128009, 128006, 882, 128007, 271, 41551, 264, 3177, 70395, 22380, 369, 264, 3130, 315, 2956, 57116, 128009, 128006, 78191, 128007, 271]


In [16]:
PHI4 = "microsoft/Phi-4-mini-instruct"
DEEPSEEK = "deepseek-ai/DeepSeek-V3.1"
QWEN_CODER = "Qwen/Qwen2.5-Coder-7B-Instruct"

In [18]:
phi4_tokenizer = AutoTokenizer.from_pretrained(PHI4)

text = "I am curiously excited to show Hugging Face Tokenizers in action to my LLM engineers"
print("Llama:")
tokens = tokenizer.encode(text)
print(tokens)
print(tokenizer.batch_decode(tokens))
print("\nPhi 4:")
tokens = phi4_tokenizer.encode(text)
print(tokens)
print(phi4_tokenizer.batch_decode(tokens))


Llama:
[128000, 40, 1097, 2917, 13610, 12304, 311, 1501, 473, 36368, 19109, 9857, 12509, 304, 1957, 311, 856, 445, 11237, 25175]
['<|begin_of_text|>', 'I', ' am', ' cur', 'iously', ' excited', ' to', ' show', ' H', 'ugging', ' Face', ' Token', 'izers', ' in', ' action', ' to', ' my', ' L', 'LM', ' engineers']

Phi 4:
[40, 939, 4396, 23138, 15209, 316, 2356, 59116, 4512, 29049, 17951, 24223, 306, 3736, 316, 922, 451, 19641, 32437]
['I', ' am', ' cur', 'iously', ' excited', ' to', ' show', ' Hug', 'ging', ' Face', ' Token', 'izers', ' in', ' action', ' to', ' my', ' L', 'LM', ' engineers']


In [19]:
print("Llama:")
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))
print("\nPhi 4:")
print(phi4_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

Llama:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Phi 4:
<|system|>You are a helpful assistant<|end|><|user|>Tell a light-hearted joke for a room of Data Scientists<|end|><|assistant|>


In [22]:
deepseek_tokenizer = AutoTokenizer.from_pretrained(DEEPSEEK)

text = "I am curiously excited to show Hugging Face Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
print(phi4_tokenizer.encode(text))
print()
print(deepseek_tokenizer.encode(text))

[128000, 40, 1097, 2917, 13610, 12304, 311, 1501, 473, 36368, 19109, 9857, 12509, 304, 1957, 311, 856, 445, 11237, 25175]

[40, 939, 4396, 23138, 15209, 316, 2356, 59116, 4512, 29049, 17951, 24223, 306, 3736, 316, 922, 451, 19641, 32437]

[0, 43, 1030, 108771, 15046, 304, 1801, 24133, 5426, 11906, 47948, 24524, 295, 4271, 304, 1026, 33792, 47, 26170]


In [26]:
print("Llama:")
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))
print("\nPhi:")
print(phi4_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))
print("\nDeepSeek:")
print(deepseek_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

Llama:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Phi:
<|system|>You are a helpful assistant<|end|><|user|>Tell a light-hearted joke for a room of Data Scientists<|end|><|assistant|>

DeepSeek:
<｜begin▁of▁sentence｜>You are a helpful assistant<｜User｜>Tell a light-hearted joke for a room of Data Scientists<｜Assistant｜></think>


In [25]:
qwen_tokenizer = AutoTokenizer.from_pretrained(QWEN_CODER)
code = """
def hello_world(person):
  print("Hello", person)
"""
tokens = qwen_tokenizer.encode(code)
for token in tokens:
  print(f"{token}={qwen_tokenizer.decode(token)}")

198=

750=def
23811= hello
31792=_world
29766=(person
982=):

220= 
1173= print
445=("
9707=Hello
497=",
1697= person
340=)

